In [1]:
import os, math, sys
from sys import stdout

import numpy as np
import matplotlib.pyplot as plt
import openmm.app  as omm_app
import openmm as omm
import simtk.unit as unit
from tqdm import tqdm
import mdtraj

### Citing: Most of the openmm data needed for butane and format of this notebook comes from [here](https://fastmbar.readthedocs.io/en/latest/butane_PMF.html).

In [2]:
# Indicates whether to run a simulation or not
SIMULATE = True

## 1. Construct an OpenMM system of butane
We set up the MD molecular system in the format/object required by OpenMM, a class called
[System](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.System.html).

We read in **psf** and **pdb** files for butane: butane.psf and butane.pdb.
The psf file, butane.psf, contains the topology of butane and it is the topolgy file format used by [CHARMM](https://www.charmm.org/).
The file used here is generated using CHARMM.
OpenMM has parsers for topology files with different format from AMBER, CHARMM, etc.

In [3]:
psf = omm_app.CharmmPsfFile('../data/butane.psf')
pdb = omm_app.PDBFile('../data/butane.pdb')
topology = psf.topology

Read CHARMM force field for butane. The CHARMM force field is downloaded from [here](http://mackerell.umaryland.edu/charmm_ff.shtml)

In [4]:
params = omm_app.CharmmParameterSet('../data/top_all35_ethers.rtf',
                                    '../data/par_all35_ethers.prm')

Create a OpenMM system based on the psf file of butane and the CHARMM force field.

In [5]:
system = psf.createSystem(params, nonbondedMethod=omm_app.NoCutoff)

After constructing the OpenMM system of butane, we can save it in an XML formated text file, which can be used later for simulations. Therefore, if we want to use the same system in another script, we can just read the text file in an OpenMM system instead of constructing it again. This file is a readable text file and can be used to see what system information is used.

In [6]:
with open("../output/system.xml", 'w') as file_handle:
    file_handle.write(omm.XmlSerializer.serialize(system))

In [7]:
## read the OpenMM system of butane
with open("../output/system.xml", 'r') as file_handle:
    xml = file_handle.read()
system = omm.XmlSerializer.deserialize(xml)

## read psf and pdb file of butane
psf = omm_app.CharmmPsfFile("../data/butane.psf")
pdb = omm_app.PDBFile('../data/butane.pdb')
topology = psf.topology

# 2. Set up and run the simulation

In order to run simulations in OpenMM, we need to construct an OpenMM [context](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html), which consists of a system, an integrator and a platform.
The system is the butane system we have constructed above.
The integrator specifies what kind integration method we should use. Here, we will use Langevin dynamics, which corresponds to the [OpenMM.LangevinIntegrator](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.LangevinIntegrator.html) and the [OpenMM.LangevinMiddleIntegrator](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.LangevinMiddleIntegrator.html). Here, we use the LangevinMiddleIntegrator based on [J. Phys. Chem. A 2019, 123, 28, 6056-6079](https://pubs.acs.org/doi/full/10.1021/acs.jpca.9b02771), closely related to the BAOAB [Proc. R. Soc. A. 472: 20160138](https://royalsocietypublishing.org/doi/abs/10.1098/rspa.2016.0138) scheme.

The platform specifies what kind of hardware we will run simulation on. Here, we choose to use CPUs because the size of the system is quite small.

Then, we initialize a [Simulation](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.app.simulation.Simulation.html), an api for running simulations and reporting results.


In [8]:
#### setup an OpenMM context
    
## platform
platform = omm.Platform.getPlatformByName('CPU')
    
## integrator
T = 800 * unit.kelvin  ## temperature
fricCoef = 10/unit.picoseconds ## friction coefficient 
stepsize = 2 * unit.femtoseconds ## integration step size
integrator = omm.LangevinMiddleIntegrator(T, fricCoef, stepsize)
simulation = omm_app.Simulation(topology, system, integrator)

We minimize the energy before running the simulations, and then add "reporters" to track various info on the simulation: the [DCD reporter](http://docs.openmm.org/7.2.0/api-python/generated/simtk.openmm.app.dcdreporter.DCDReporter.html#) records positions of atoms at each 100 steps (in a compressed format), and the [StateDataReporter](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.app.statedatareporter.StateDataReporter.html#) records the potential energy at each 100 steps.

Then, we simulate.

In [9]:
my_state = "../output/traj/state_hightemp.txt"
my_dcd = "../output/traj/traj_hightemp.dcd"

if SIMULATE:
    simulation.context.setPositions(pdb.positions)
    if psf.boxVectors is not None:
        print("we got box vectors!")
        simulation.context.setPeriodicBoxVectors(*psf.boxVectors)
    simulation.minimizeEnergy()
    
    simulation.reporters.append(omm_app.DCDReporter(my_dcd, int(1E2)))
    simulation.reporters.append(omm_app.StateDataReporter(my_state, 1E2, step=True, potentialEnergy=True))
    simulation.reporters.append(omm_app.StateDataReporter(stdout, 1E6, step=True, potentialEnergy=True))
    simulation.step(1E6)

## 3. RMSD-align trajectory frames to normalize out rotations and translations

The command [superpose](https://mdtraj.org/1.9.4/api/generated/mdtraj.Trajectory.superpose.html) rotates/translates each frame of a trajectory (modifes `traj' object in place) to minimize the RMSD to a particular reference frame. The operation specifically used to find the RMSD between two frames is [here](https://github.com/mdtraj/mdtraj/blob/275102dc3e1a1b58f83366f1459ebab9fe575889/mdtraj/rmsd/src/theobald_rmsd.cpp) in the documentation for mdtraj.

Here, we compute the butane dihedrals and
save them in csv files. The dihedral is between four atoms with indices of 3, 6, 9, and 13. 
Here we use the Python package [mdtraj](https://mdtraj.org/) to compute dihedrals.

In [10]:
topology = mdtraj.load_psf("../data/butane.psf")
traj = mdtraj.load(my_dcd, top=topology)

# RMSD-align trajectories to the a frame in the trajectory
traj.superpose(traj)
my_aligned_dcd = "../output/traj/traj_aligned_hightemp.dcd"
traj.save_dcd(my_aligned_dcd)

topology = mdtraj.load_psf("../data/butane.psf")
aligned_traj = mdtraj.load(my_aligned_dcd, top=topology)



OSError: No such file: ../output/traj/traj_hightemp.dcd

# 4. Compute dihedral angle

Here, we compute the butane dihedrals and
save them in csv files. The dihedral is between four atoms with indices of 3, 6, 9, and 13. 
Here again we use the Python package [mdtraj](https://mdtraj.org/) to compute dihedrals.

In [ ]:
theta = mdtraj.compute_dihedrals(aligned_traj, [[3, 6, 9, 13]])
np.savetxt(f"../output/dihedral/dihedral.csv", theta, fmt = "%.5f", delimiter = ",")

dihedrals = np.loadtxt( f"../output/dihedral/dihedral.csv", delimiter = ",")
_ = plt.hist(dihedrals, bins=50, density=True)

: 

Select just the atoms used for the dihedral and keep their positions in a numpy array, as well as the potential and temperature for the trajectory

In [ ]:
aligned_traj_backbone = aligned_traj.atom_slice([3, 6, 9, 13])
aligned_traj_backbone_coords = aligned_traj_backbone.xyz
N = aligned_traj_backbone_coords.shape[0]
data = aligned_traj_backbone_coords.reshape(N, 12)

potential = np.loadtxt("../output/traj/state_hightemp.txt", delimiter=',')[:, 1]
kbT = unit.BOLTZMANN_CONSTANT_kB * T * unit.AVOGADRO_CONSTANT_NA 
kbT = kbT.value_in_unit(unit.kilojoule_per_mole)
fname = "../output/butane.npz"
np.savez(fname, data=data, dihedrals=dihedrals, potential=potential, kbT=kbT)

: 

In [ ]:
potential_stuff = np.loadtxt("../output/traj/state_hightemp.txt", delimiter=',')
gibbs = np.exp(-potential_stuff/(kbT))
plt.scatter(dihedrals[::1], gibbs[::1, 1], s=1.0)
plt.xlabel("dihedral angle")
plt.ylabel("gibbs probability")
plt.ylim([0, 1E-7])

: 